# Preparação dos Dados

## 1. Seleção dos Dados


### 1.1 Dados a Incluir

### 1.2 Dados a Excluir

## 2. Limpeza dos Dados

## 3. Construção de Dados

### 3.1 Atributos Derivados

## 4. Integração de Dados

## 5. Formatação de Dados 